In [68]:
import tensorflow as tf
import numpy as np
import random

In [69]:
def get_data(set_type):
    """Get data from files and storage them in a array. Return the data_set and label_set.
    
    set_type    the type of data set you want to build, including train dataset, dev dataset 
                and eval dataset
    """
    
    data_path = {'train': 'train/lab/hw1train_labels.txt'} #find the label file path needed for building dataset
    path_prefix = '/Users/changsongdong/Dropbox/UIUC/ECE544/hw/hw1/'
    full_path = path_prefix + data_path[set_type] #find the full path of the label file
    label_array = np.loadtxt(full_path, dtype='string') #load the label file into a array

    #creat empty arrays to insert label and data
    label_set = np.zeros(len(label_array))
    data_set = np.zeros([len(label_array), 16])
    #the first column of the label file is the label,
    #the second column is the corresbonding data file nam
    for i in range(len(label_array)): 
        #build the label set
        label_set[i] = label_array[i][0] #insert label into label_set
        #build the data set
        with open(path_prefix + label_array[i][1]) as data_file:
            data = data_file.readlines()[0].split() #find the data accoding to label
        for j in range(len(data)):
            data_set[i][j] = data[j] #insert data into the dataset
            
    data_set, label_set = nan_check(data_set, label_set) #delete the rows containing 'nan'

    return data_set, label_set #return the data set and label set


In [70]:
def nan_check(data, label):
    """Find out the rows in datasets and delete these rows
    
    """
    
    nan_rows = np.array(0); #define an array containg the no. of rows having 'nan'
    
    #collect all the numbers of 'nan'-data rows
    for i in range(len(data)):
        for j in range(16):
            if str(data[i][j]) == 'nan':
                nan_rows = np.append(nan_rows, i)
    nan_rows = np.delete(nan_rows, 0) #delete the first element of nan_rows which was made to fit the append()
    
    return np.delete(data, nan_rows, 0), np.delete(label, nan_rows) #output the dataset whose 'nan'-data rows have been deleted


In [71]:
def shuffle(data_set, label_set):
    """Randomly shuffle the data and label
    
    data_set    the data samples
    
    label_set   the lables
    """
    
    shuffled_data = np.zeros((data_set.shape))
    shuffled_label = np.zeros((len(label_set)))
    idx = np.array(xrange(len(label_set)))
    random.shuffle(idx)
    i = 0
    for j in idx:
        shuffled_data[i] = data_set[int(j)]
        shuffled_label[i] = label_set[int(j)]
        i += 1
    return shuffled_data, shuffled_label

In [72]:
train_data, train_label = get_data('train')
train_data, train_label = shuffle(train_data, train_label)

label = np.zeros((8774, 1))
for i in range(len(train_label)):
    label[i][0] = train_label[i]
print label.shape


(8774, 1)


In [73]:
x_placeholder = tf.placeholder(tf.float32, [None, 16])
y_placeholder = tf.placeholder(tf.float32, [None, 1])
w = tf.Variable(tf.random_normal([16, 1]))
b = tf.Variable(tf.zeros([1]))
y_hat = tf.matmul(x_placeholder, w) + b
loss = tf.reduce_sum(tf.square(y_placeholder-y_hat))

train_step = tf.train.GradientDescentOptimizer(0.00000001).minimize(loss)

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for step in range(100):
    feed_dict = {x_placeholder: train_data, y_placeholder: label}
    sess.run(train_step, feed_dict=feed_dict)
    if step % 5 == 0:
        print("Loss at step %d: %f" % (step, loss_np))
print sess.run(w)
w_np = sess.run(w)

Loss at step 0: 5262.000000
Loss at step 5: 5262.000000
Loss at step 10: 5262.000000
Loss at step 15: 5262.000000
Loss at step 20: 5262.000000
Loss at step 25: 5262.000000
Loss at step 30: 5262.000000
Loss at step 35: 5262.000000
Loss at step 40: 5262.000000
Loss at step 45: 5262.000000
Loss at step 50: 5262.000000
Loss at step 55: 5262.000000
Loss at step 60: 5262.000000
Loss at step 65: 5262.000000
Loss at step 70: 5262.000000
Loss at step 75: 5262.000000
Loss at step 80: 5262.000000
Loss at step 85: 5262.000000
Loss at step 90: 5262.000000
Loss at step 95: 5262.000000
[[-0.52567858]
 [-0.52282906]
 [-0.44088125]
 [ 0.66628277]
 [-0.16787408]
 [ 0.19038929]
 [-1.37779593]
 [-0.42531368]
 [ 0.05143901]
 [ 0.31047881]
 [-0.43625134]
 [ 0.82192385]
 [ 0.44409898]
 [-1.17862022]
 [ 0.45010945]
 [-0.00670841]]
